# Laboratoire 3 : Machines à vecteurs de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             |                                                         |
|-----------------------|---------------------------------------------------------|
| Jean-Philippe Decoste |  DECJ19059105                                           |
| Ahmad Al-Taher        |   ALTA22109307                                          |
| Stéphanie Lacerte     |   LACS06629109                                          |
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 02                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 30 oct 2018                                             |

In [2]:
import csv
import math
import os

import graphviz
import matplotlib.pyplot as plt
import numpy as np
from imageV2 import Image as imageFeat
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, ShuffleSplit, StratifiedShuffleSplit

from spam import Spam
import utilities
from tabulate import tabulate

In [3]:
def prepareDataset(datasetName, dataset):
    print("PREPARING DATASETS")
    allData_length = len(list(csv.reader(open(dataset))))
    progress = 0
    datas = []

    print("Reading " + datasetName + " features:")
    utilities.printProgressBar(0, allData_length, prefix='Progress:', suffix='Complete', length=50)
    with open(dataset, 'r') as theFile:
        primitives = csv.reader(theFile, delimiter=',', quotechar='|')

        for row in primitives:
            progress += 1
            utilities.printProgressBar(progress+1, allData_length, prefix='Progress', suffix='Complete', length=50)

            values = [float(i) for i in row]
            if datasetName == "Galaxy":
                datas.append(imageFeat(values))
            elif datasetName == "Spam":
                datas.append(Spam(values))
    print("\n-> Done\n")

    #3. Split dataset using model_selection
    for data in np.array(datas):
        features.append(data.features)
        answers.append(data.answer)

## SVM

## Réseaux neuronaux

## Main

In [5]:
#1.A Read Galaxy features (name of file, path, n_split, test size, random state)
features = []
answers = []

prepareDataset("Galaxy", r"galaxies.csv")



PREPARING DATASETS
Reading Galaxy features:
Progress |**************************************************| 100.0% Complete
-> Done

[43.930924087036736, 51.01459655976993, 57.710347821368494, 10.04080181393207, 11.753778539411638, 12.423907531431968, 20.01964231316385, 20.145107092058197, 23.890885542842724, 18.698736310974507, 15.543446679530906, 17.429573092368198, 3.860164803435622, 3.685242829417392, 3.8499784988736665, 0.35608710094600127, 0.5968399839318662, 0.5457711835289252, 0.5062480342359442, 171.400450796202, 86.77114128273193, 2849625.0, 7.1205854415893555, 0.30214676507437943, 243.95547057235152, 0.0841936632253471, 3.403742192024813, 0.2241284442662666, 0.9342236609188999, 1.0, -0.26520337421586726, 0.9519712600246981, 1.159898581793148, 0.3147050654297465, 0.917173933388988, 1.0235222460688695, 0.21882561074248708, 0.24613368959657067, 0.2686791152083071, 0.9057091736506169, 0.910898956929118, 0.2500723507154444, 113.0, 168.0, 214.0, 60.54944071588367, 70.83149888143177,

## Introduction

## Question 1 
### Présentation de la méthode de validation

## Question 2
### Description du modèle élaboré

## Question 3
### Analyse de la phase d'entraînement

## Question 4
### Matrice des expérimentations

### Étude des hyperparamètres des modèles

## Question 5
### Matrice des expérimentations

### Études des hyperparamètres des modèles

## Question 6
### Discussion de l'impact de la taille des données

## Question 7
### Discussion de l'impact du bruit

## Question 8
### Formulation des recommandations

## Question 9
### Améliorations possibles

## Conclusion

## Bibliographie